In [2]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
import geopandas as gpd
import xdem
import rasterio
DEM_folder = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/'

In [4]:
# make a vrt of raw tiles
vrt = gdal.BuildVRT(f'{DEM_folder}vrt_and_derivates/DEM_GER.vrt', getFilelist(f'{DEM_folder}raw_tiles/', ',tif'), separate = False)
vrt = None
convertVRTpathsTOrelative(f'{DEM_folder}vrt_and_derivates/DEM_GER.vrt')

vrt = gdal.BuildVRT(f'{DEM_folder}vrt_and_derivates/LAT_GER.vrt', getFilelist(f'{DEM_folder}Force_Tiles/LAT/', ',tif'), separate = False)
vrt = None
convertVRTpathsTOrelative(f'{DEM_folder}vrt_and_derivates/LAT_GER.vrt')

vrt = gdal.BuildVRT(f'{DEM_folder}vrt_and_derivates/LON_GER.vrt', getFilelist(f'{DEM_folder}Force_Tiles/LON/', ',tif'), separate = False)
vrt = None
convertVRTpathsTOrelative(f'{DEM_folder}vrt_and_derivates/LON_GER.vrt')


In [ ]:
# load shp to get projection and warp DEM vrt to match FORCE porjection
force_grid = gpd.read_file('/data/Aldhani/eoagritwin/misc/FORCE_footprint/datacube-grid_DEU_10km.gpkg')
ext = force_grid.total_bounds
# manually change the first value due to rounding bug
ext[0] = ext[0] - .25
gt = (float(ext[0] - 30000), 20.0, 0.0, float(ext[3]) + 30000, 0.0, -20.0) # also extend, so that slope and aspect won't have a small gap at borders
prj = 'PROJCS["ETRS89-extended / LAEA Europe",GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4258"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",52],PARAMETER["longitude_of_center",10],PARAMETER["false_easting",4321000],PARAMETER["false_northing",3210000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Northing",NORTH],AXIS["Easting",EAST],AUTHORITY["EPSG","3035"]]'
cols = (np.unique(np.asarray(force_grid.bounds['minx'])).shape[0] + 2) * 1500
rows = (np.unique(np.asarray(force_grid.bounds['miny'])).shape[0] + 2) * 1500
out_ds = gdal.GetDriverByName('GTiff').Create(f'{DEM_folder}reprojected/FORCE_TILE_DUMMY.tif', cols, rows, 1, gdal.GDT_Int8)
out_ds.SetGeoTransform(gt)
out_ds.SetProjection(prj)
out_ds.GetRasterBand(1).WriteArray(np.random.randint(-120, 121, size=(rows, cols)))
del out_ds

In [ ]:
warp_raster_to_reference(f'{DEM_folder}vrt_and_derivates/DEM_GER.vrt',
                         f'{DEM_folder}reprojected/FORCE_TILE_DUMMY.tif',
                         f'{DEM_folder}reprojected/DEM_GER_FORCE_WARP.tif')

Raster warped and saved to: /data/Aldhani/eoagritwin/et/Auxiliary/DEM/vrt_and_derivates/DEM_GER_FORCE_WARP.tif


In [3]:
# calculate slope and aspect
dem = xdem.DEM(f'{DEM_folder}reprojected/DEM_GER_FORCE_WARP.tif')
slope = dem.slope()
aspect = dem.aspect()


In [4]:
opts = {
    "BIGTIFF": "YES",
    "COMPRESS": "DEFLATE", 
    "TILED": "YES"
}

slope.save(f'{DEM_folder}reprojected/SLOPE_GER_FORCE_WARP.tif', co_opts=opts)
aspect.save(f'{DEM_folder}reprojected/ASPECT_GER_FORCE_WARP.tif', co_opts=opts)

In [7]:
# now cut DEM, aspect, slope into FORCE TILES
force_grid = gpd.read_file('/data/Aldhani/eoagritwin/misc/FORCE_footprint/datacube-grid_DEU_10km.gpkg')

for product, name in zip([f'{DEM_folder}reprojected/DEM_GER_FORCE_WARP.tif', 
                          f'{DEM_folder}reprojected/SLOPE_GER_FORCE_WARP.tif',
                          f'{DEM_folder}reprojected/ASPECT_GER_FORCE_WARP.tif'],
                          ['DEM', 'SLOPE', 'ASPECT']):
    with rasterio.open(product) as src:
        varPath = f'{DEM_folder}Force_Tiles/{name}'
        if not os.path.exists(varPath):
            os.makedirs(varPath)
        for tileID in force_grid['Tile_ID']:
            tile = force_grid[force_grid['Tile_ID'] == tileID]
            # print(Vector.crs)
            out_image, out_transform=rasterio.mask.mask(src,tile.geometry,crop=True)
            out_meta=src.meta.copy() # copy the metadata of the source DEM
            
            out_meta.update({
                "driver":"Gtiff",
                "height":out_image.shape[1], # height starts with shape[1]
                "width":out_image.shape[2], # width starts with shape[2]
                "transform":out_transform
            })
                    
            with rasterio.open(f'{varPath}/{name}_{tileID}.tif','w',**out_meta) as dst:
                dst.write(out_image)

In [ ]:
# warp DEM, SLOPE, ASPECT, LAT and LON to LST for air temp correction

warp_raster_to_reference(f'{DEM_folder}vrt_and_derivates/DEM_GER.vrt',
                         '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/daily_observations_all/2019/Daily_LST_max_2019_April.tif',
                         f'{DEM_folder}reprojected/DEM_GER_LST_WARP.tif')

warp_raster_to_reference(f'{DEM_folder}reprojected/ASPECT_GER_FORCE_WARP.tif',
                         '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/2019/Daily_AcqTime_maxLST_2019_April.tif',
                         f'{DEM_folder}reprojected/ASPECT_GER_LST_WARP.tif')

warp_raster_to_reference(f'{DEM_folder}reprojected/ASPECT_GER_FORCE_WARP.tif',
                         '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/2019/Daily_AcqTime_maxLST_2019_April.tif',
                         f'{DEM_folder}reprojected/ASPECT_GER_LST_WARP.tif')

warp_raster_to_reference(f'{DEM_folder}reprojected/SLOPE_GER_FORCE_WARP.tif',
                         '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/2019/Daily_AcqTime_maxLST_2019_April.tif',
                         f'{DEM_folder}reprojected/SLOPE_GER_LST_WARP.tif')

warp_raster_to_reference(f'{DEM_folder}vrt_and_derivates/LAT_GER.vrt',
                         '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/2019/Daily_AcqTime_maxLST_2019_April.tif',
                         f'{DEM_folder}reprojected/LAT_GER_LST_WARP.tif')

warp_raster_to_reference(f'{DEM_folder}vrt_and_derivates/LON_GER.vrt',
                         '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/2019/Daily_AcqTime_maxLST_2019_April.tif',
                         f'{DEM_folder}reprojected/LON_GER_LST_WARP.tif')

Raster warped and saved to: /data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/ASPECT_GER_LST_WARP.tif
Raster warped and saved to: /data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/SLOPE_GER_LST_WARP.tif
Raster warped and saved to: /data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LAT_GER_LST_WARP.tif
Raster warped and saved to: /data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LON_GER_LST_WARP.tif
